In [3]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

# 1. 데이터 가져오기


In [2]:
import pandas as pd
df = pd.read_csv("C:/ai_x/download/ShareData/부동산_250213/최종전국평당분양가격(결측치제외).csv",encoding='cp949')
df.head()

,지역명,연도,월,평당분양가격
0,서울,2013,12,18189.0
1,부산,2013,12,8111.0
2,대구,2013,12,8080.0
3,인천,2013,12,10204.0
4,광주,2013,12,6098.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역명     2176 non-null   object 
 1   연도      2176 non-null   int64  
 2   월       2176 non-null   int64  
 3   평당분양가격  2176 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 68.1+ KB


In [51]:
df

,지역명,연도,월,평당분양가격,지역명2n,연도n,월n,연도s,월s
0,서울,2013,12,18189.0,8,0.0,1.000000,-1.875367,1.62196
1,부산,2013,12,8111.0,7,0.0,1.000000,-1.875367,1.62196
2,대구,2013,12,8080.0,5,0.0,1.000000,-1.875367,1.62196
3,인천,2013,12,10204.0,11,0.0,1.000000,-1.875367,1.62196
4,광주,2013,12,6098.0,4,0.0,1.000000,-1.875367,1.62196
...,...,...,...,...,...,...,...,...,...
2171,전북,2024,8,12058.2,13,1.0,0.636364,1.664199,0.46374
2172,전남,2024,8,13120.8,12,1.0,0.636364,1.664199,0.46374
2173,경북,2024,8,13827.0,3,1.0,0.636364,1.664199,0.46374
2174,경남,2024,8,13252.8,2,1.0,0.636364,1.664199,0.46374


# 2. 지역명의 라벨 인코딩
- 지역명을 라벨인코딩한 지역명 2 필드 추가 
- 분석할 경우 원핫인코딩까지 할 것을 추천


In [ ]:
'''
1. 지역명만 numpy로 추출
2. 추출된 지역명 라벨인코딩
3. 라벨인코딩 데이터프레임으로 변경
4. df에 지역명2n으로 저장하고 재할당
'''

In [16]:
from tensorflow.keras.utils import to_categorical # 분류분석시 원핫인코딩 (전처리 작업) 
from tensorflow.keras.models import Sequential    # 모델 생성
from tensorflow.keras.layers import Dense,Input 
from sklearn.preprocessing import LabelEncoder    # 라벨인코딩
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd

# 지역명만 numpy로 추출
new_city = df['지역명'].values
print('원 데이터 :', new_city)

# 추출된 지역명 라벨인코딩 작업
le = LabelEncoder()  # 라벨인코딩 객체 생성
le.fit(new_city)
labeled_data = le.fit_transform(new_city)
print('라벨인코딩된 데이터 :\n', labeled_data)
# one_hot_encoded_data = to_categorical(labeled_data)
# print('원핫인코딩된 데이터:\n', one_hot_encoded_data)

#라벨인코딩 데이터(labeled_data)를 데이터 프레임으로 변경
new_df = pd.DataFrame(labeled_data)
new_df

#df에 지역명2n으로 저장하기
df['지역명2']=new_df
# df['지역명2'] = le.fit_transform(df['지역명']) --> 이렇게 하는방법을 추천(선생님) / tolist, pd.Series 로 할 수도 있음

원 데이터 : ['서울' '부산' '대구' ... '경북' '경남' '제주']
라벨인코딩된 데이터 :
 [ 8  7  5 ...  3  2 14]


# 3. normalization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(MinMaxScaler이용)
- 지역명 2n, 연도n, 월n 필드 추가 

## 입력변수('지역명2n','연도','월') 스케일 조정 & 타겟변수('평당분양가격') 스케일 조정

In [32]:
from sklearn.preprocessing import MinMaxScaler 

x_data = df[['연도','월','지역명2']].values # 독립변수가 3개라서 x(소문자x)가 아니라 X(대문자X)를 쓰는게 대부분임
y_data = df[['평당분양가격']].values

#x_data 스케일 조정
scaler_x = MinMaxScaler() # x_data를 정규화 시킬 객체 
scaled_x_data = scaler_x.fit_transform(x_data)
scaled_x_data

#y_data 스케일 조정 
scaler_y = MinMaxScaler()
scaled_y_data = scaler_y.fit_transform(y_data)
scaled_y_data

#x_data 데이터 프레임으로 변환 후 연도n, 월n 필드 추가 
new_x = pd.DataFrame(scaled_x_data)
new_x 
df['지역명2n'] =new_x.iloc[:,-1:]
df['연도n'] = new_x.iloc[:,:1]
df['월n'] = new_x.iloc[:,1:2]


#y_data 데이터 프레임으로 변환 후 평당분양가격n 필드 추가
new_y = pd.DataFrame(scaled_y_data)
df['평당분양가격n'] = new_y.iloc[:,:1]

# 4. standardization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(StandardScaler이용)
- 지역명 2s, 연도s, 월s필드 추가 

In [35]:
from sklearn.preprocessing import StandardScaler

x_data = df[['연도','월','지역명2']].values
x_data
y_data = df[['평당분양가격']].values
y_data


scaler_x = StandardScaler() #x_data를 표준화할 객체
scaler_y = StandardScaler() #y_data를 표준화할 객체 
scaled_x_data = scaler_x.fit_transform(x_data)
scaled_y_data = scaler_y.fit_transform(y_data)
scaled_y_data


#x_data 필드 추가 
news_x = pd.DataFrame(scaled_x_data)
news_y = pd.DataFrame(scaled_y_data)
df['지역명2s'] = new_x.iloc[:,-1:]
df['연도s'] = news_x.iloc[:,:1]
df['월s'] = news_x.iloc[:,1:2]



df['평당분양가격s'] = news_y.iloc[:,:1]

In [4]:
import pandas as pd
df = pd.read_csv('data/test_scale.csv')

In [38]:
df.sample(5)

,지역명,연도,월,평당분양가격,지역명2n,연도n,월n,연도s,월s,평당분양가격n,평당분양가격s,지역명2,지역명2s
1894,울산,2023,4,18423.9,0.6250,0.909091,0.272727,1.342421,-0.694479,0.334326,1.212805,10,0.6250
1106,인천,2019,6,12827.1,0.6875,0.545455,0.454545,0.055306,-0.115370,0.188312,0.159363,11,0.6875
2174,경남,2024,8,13252.8,0.1250,1.000000,0.636364,1.664199,0.463740,0.199418,0.239489,2,0.1250
361,대구,2015,10,8850.6,0.3125,0.181818,0.818182,-1.231809,1.042850,0.084570,-0.589103,5,0.3125
1572,세종,2021,9,12553.2,0.5625,0.727273,0.727273,0.698863,0.753295,0.181166,0.107809,9,0.5625


# 5. 지역명을 원핫인코딩

In [39]:
one_hot_encoded_data = to_categorical(df['지역명2'])
print('원핫인코딩된 데이터:\n', one_hot_encoded_data) # 희소행렬 


원핫인코딩된 데이터:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [49]:
# 지역명, 지역명2 # 데이터 프레임 sort.values
loc_lifo = df[['지역명','지역명2']].head(17).sort_values(by='지역명2') # pandas의 get_dummies 써도 됨
loc_column_names = loc_lifo['지역명'].tolist()
print(loc_column_names)

['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']


In [54]:
df[loc_column_names] = to_categorical(df['지역명2']).astype(np.int16)
df.head()

,지역명,연도,월,평당분양가격,지역명2n,연도n,월n,연도s,월s,평당분양가격n,평당분양가격s,지역명2,지역명2s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,0.5000,0.0,1.0,-1.875367,1.62196,0.328198,1.168591,8,0.5000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,0.4375,0.0,1.0,-1.875367,1.62196,0.065274,-0.728312,7,0.4375,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,0.3125,0.0,1.0,-1.875367,1.62196,0.064466,-0.734147,5,0.3125,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,0.6875,0.0,1.0,-1.875367,1.62196,0.119878,-0.334363,11,0.6875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,0.2500,0.0,1.0,-1.875367,1.62196,0.012757,-1.107203,4,0.2500,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
pd.options.display.max_columns = 30 # 최대 출력가능한 데이터프레임 열수 (기본 20)
df.head()

,지역명,연도,월,평당분양가격,지역명2n,연도n,월n,연도s,월s,평당분양가격n,평당분양가격s,지역명2,지역명2s,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
0,서울,2013,12,18189.0,0.5000,0.0,1.0,-1.875367,1.62196,0.328198,1.168591,8,0.5000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,부산,2013,12,8111.0,0.4375,0.0,1.0,-1.875367,1.62196,0.065274,-0.728312,7,0.4375,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,대구,2013,12,8080.0,0.3125,0.0,1.0,-1.875367,1.62196,0.064466,-0.734147,5,0.3125,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,인천,2013,12,10204.0,0.6875,0.0,1.0,-1.875367,1.62196,0.119878,-0.334363,11,0.6875,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,광주,2013,12,6098.0,0.2500,0.0,1.0,-1.875367,1.62196,0.012757,-1.107203,4,0.2500,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
